In [1]:
import spiceypy as spice
import numpy as np

## Kernels Loading

In [ ]:
# Load the necessary SPICE kernels
spice.furnsh("./kernels/naif0012.tls")  # Load a leapseconds kernel
spice.furnsh("./kernels/de440s.bsp")
spice.furnsh("./kernels/pck00011.tpc")
spice.furnsh("./kernels/gm_de440.tpc")
spice.furnsh("./kernels/earth_200101_990827_predict.bpc")
spice.furnsh("./kernels/earth_assoc_itrf93.tf")
spice.furnsh("./kernels/moon_assoc_me.tf")
# spice.furnsh("./kernels/moon_assoc_pa.tf") # only me or pa, not both
spice.furnsh("./kernels/moon_de440_220930.tf")
spice.furnsh("./kernels/moon_pa_de440_200625.bpc")

## Time Conversion

Common Time Formats: -
- Ephemeris Time (ET): A continuous time scale used for spacecraft and astronomical events.
- UTC (Coordinated Universal Time): A time standard that combines Universal Time (UT) and atomic time.
- TDB (Barycentric Dynamical Time): A time scale used for solar system dynamics.
- TT (Terrestrial Time): A time standard closely related to TAI (International Atomic Time).

In [3]:
# 1. Convert UTC to Ephemeris Time (ET)
utc_time = '2023-01-01T00:00:00'  # ISO format
et_time = spice.utc2et(utc_time)
print(f'UTC: {utc_time} -> ET: {et_time}')

# 2. Convert Ephemeris Time (ET) to UTC
utc_converted = spice.et2utc(et_time, 'ISOC', 2)
print(f'ET: {et_time} -> UTC: {utc_converted}')

# 3. Convert ET to TDB
# tdb_time = spice.et2tdb(et_time)
# print(f'ET: {et_time} -> TDB: {tdb_time}')

# 4. Convert TDB to ET
# et_from_tdb = spice.tdb2et(tdb_time)
# print(f'TDB: {tdb_time} -> ET: {et_from_tdb}')

# 5. Convert ET to TT (Terrestrial Time)
# tt_time = spice.et2tt(et_time)
# print(f'ET: {et_time} -> TT: {tt_time}')

# 6. Convert TT to ET
# et_from_tt = spice.tt2et(tt_time)
# print(f'TT: {tt_time} -> ET: {et_from_tt}')

UTC: 2023-01-01T00:00:00 -> ET: 725803269.1839136
ET: 725803269.1839136 -> UTC: 2023-01-01T00:00:00.00


## Frames Transformation
Common Coordinate Frames: -
- J2000: An inertial frame based on the position of the Earth at the epoch J2000 (January 1, 2000).
- ECLIPJ2000: An ecliptic coordinate frame based on the Earth's orbit around the Sun.
- MOON_ME: A mean equatorial frame centered on the Moon.
- MOON_PA: A pole-aligned frame centered on the Moon.

In [4]:
# Transform coordinates from one reference frame to another
# Example: Transforming from J2000 to ECLIPJ2000
position_j2000          = [1.0, 2.0, 3.0]  # Example coordinates in J2000
transformation_matrix   = spice.pxform('J2000', 'ECLIPJ2000', spice.utc2et('2023-01-01T00:00:00'))
position_ecliptic       = spice.mxv(transformation_matrix, position_j2000)

## Position and Velocity 

In [5]:
# Get the position and velocity of Mars from Earth at a specific time
et          = spice.utc2et('2023-01-01T00:00:00') # or spice.str2et("2023-01-01T00:00:00.000")

# Compute the state (position and velocity) of the spacecraft relative to the observer
state, ltime    = spice.spkezr("MOON", et, "J2000", "NONE", "EARTH")
position        = state[0:3]  # Position vector
velocity        = state[3:6]  # Velocity vector

# Output the results
print("Position (km):", position)
print("Velocity (km/s):", velocity)

Position (km): [325449.6968924  198317.20939396  80622.99408606]
Velocity (km/s): [-0.50488315  0.75955089  0.4249488 ]


## Light-Time Correction

In [6]:
# Assume we want to calculate the light-time from Earth to Mars
et_earth    = spice.utc2et('2023-01-01T00:00:00')
et_moon     = spice.utc2et('2023-01-01T00:05:00')  # Example time 5 minutes later

# Direct should be set to 'FROM' for light-time from the observer to the target
# Targ is the target body (e.g., 'MOON')
light_time  = spice.ltime(et_moon, et_earth, 'FROM', 'MOON')
print(f'Light-time: {light_time} seconds')

TypeError: 'float' object cannot be interpreted as an integer

## Observation Geometry

In [7]:
# Find the sub-point on Mars from the position of the spacecraft
spacecraft_pos  = [1.0, 2.0, 3.0]  # Example spacecraft position in ECLIPJ2000
subpoint        = spice.subpt('Intercept', 'MOON', et, spacecraft_pos)
print(f'Sub-point on Moon: {subpoint}')


TypeError: subpt() missing 1 required positional argument: 'obsrvr'

## Rotation and Orientation

In [ ]:
# Determine the rotation from J2000 to ITRF93
et              = spice.utc2et('2023-01-01T00:00:00')
rotation_matrix = spice.pxform('J2000', 'ITRF93', et) # from J2000 to ITRF93
print(rotation_matrix)

[[-1.75307269e-01  9.84513702e-01  3.62822322e-04]
 [-9.84511281e-01 -1.75307643e-01  2.18365552e-03]
 [ 2.21344431e-03  2.56080185e-05  9.99997550e-01]]


## Planetary Ephemeris

In [ ]:
# Retrieve the state of Moon with respect to Earth
et              = spice.utc2et('2023-01-01T00:00:00')
venus_state     = spice.spkezr('MOON', et, 'J2000', 'NONE', 'EARTH') # state vector from Earth to Moon
print(f'Venus State: {venus_state}')

Venus State: (array([ 3.25449697e+05,  1.98317209e+05,  8.06229941e+04, -5.04883150e-01,
        7.59550892e-01,  4.24948798e-01]), 1.2993907357498393)


## Clearing SPICE kernels

In [ ]:
# Unload the kernels after use
spice.kclear()